In [1]:
import os
import re

import cv2
import matplotlib.pyplot as plt
import numpy as np

from scipy.spatial.distance import cdist

%matplotlib inline

In [2]:
ANNOTATIONS_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/480p/"
CONTOURS_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/480p/"
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'
CLOSING_KERNEL_SIZE = 25

In [3]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [4]:
def load_gray_img(img_path):
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img_gray

In [5]:
def close_image(image, closing_kernel_size):
    '''Returns the image that is closed with a elliptical kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(closing_kernel_size, closing_kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [6]:
def extract_longest_contour(image, closing_kernel_size, method):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    image_closed = close_image(image, closing_kernel_size)
    
    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(image_closed, 127, 255, 0)

    # Find contour
    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    contours, _ = cv2.findContours(image=thresh,
                                   mode=cv2.RETR_TREE,
                                   method=method)
    
    # Get longest contour from contours
    longest_contour = max(contours, key=len).astype(np.float32)
    
    return longest_contour

In [7]:
def get_translations(contour_0, contour_1):
    '''Returns the translations for each point in contour_0 to contour_1.'''
    
    contour_0 = np.squeeze(contour_0)
    contour_1 = np.squeeze(contour_1)
    
    #print(contour_0.shape, contour_1.shape)
    
    translations = np.empty([0, 2], int)
    
    for contour_point_0, contour_point_1 in zip(contour_0, contour_1):
        translation = np.subtract(contour_point_1, contour_point_0)
        translations = np.append(translations, translation.reshape([1,2]), axis=0)
    
    return translations

In [8]:
def save_annotation_with_contour_and_translation(annotation, contour, translation, path):
    '''Saves annotation plotted with contour and translation to a given path.'''
    
    plt.figure(figsize=(15,10))
    
    # Plot contour
    contour = np.squeeze(contour)
    plt.scatter(contour[:, 0], contour[:, 1])
    
    # Plot annotation
    plt.imshow(annotation)
    
    # Plot translation
    for c, t in zip(contour, translation):
        plt.arrow(c[0], c[1],
                  t[0], t[1],
                  width=1, color='r')  
    
    # Save image
    plt.savefig(path, bbox_inches=0)    
    plt.close()

In [9]:
def find_closest(contour_point_1, contour_1_real):
    contour_1_real = np.squeeze(contour_1_real, axis = 1)
    dist = cdist(contour_point_1, contour_1_real, metric='euclidean')
    arg_contour_point_1_final = np.argmin(dist)
    contour_point_1_final = contour_1_real[arg_contour_point_1_final]
    #print('contour_point_1:', contour_point_1, '--> contour_point_1_final:', contour_point_1_final)
    return contour_point_1_final
    
def match_points(contour_1, contour_1_real):
    '''Match points based on minimal distance. 2 points can be matched to the same point if it is the closest'''
    rows_1, _, _ = contour_1.shape
    contour_1_final = np.zeros((rows_1, 2))
    for x in range(rows_1):
        contour_1_final[x] = find_closest(contour_1[x], contour_1_real)
    return contour_1_final

In [10]:
def create_contours_and_translations(annotations_folders_path,
                                     contours_folders_path,
                                     translations_folders_path,
                                     closing_kernel_size):

    # Get list of sequences
    sequences = os.listdir(annotations_folders_path)
    sequences.sort()
    
    
    # Iterate through sequences
    for i, sequence in enumerate(sequences):

        # Debug
        if (i > 0): break

        print('#{}: {}'.format(i, sequence))

        # Create folder to save Contours
        contours_folder_path = os.path.join(contours_folders_path, sequence)
        if not os.path.exists(contours_folder_path):
            os.makedirs(contours_folder_path)        

        # Create folder to save Translations
        translations_folder_path = os.path.join(translations_folders_path, sequence)
        if not os.path.exists(translations_folder_path):
            os.makedirs(translations_folder_path)

        # Get list of frames
        frames = os.listdir(os.path.join(annotations_folders_path, sequence))
        if '.ipynb_checkpoints' in frames:
            frames.remove('.ipynb_checkpoints')
        frames.sort()

        # Iterate through frames
        for j, frame in enumerate(frames):

            # Debug
            #if (j > 0): break
            print('\t#{}: {}'.format(j, frame))

            # Get path to frames
            frame_0_path = os.path.join(annotations_folders_path, sequence, frame)
            try:
                frame_1_path = os.path.join(annotations_folders_path, sequence, frames[j+1])
            # Break if frame_0 is last frame
            except IndexError as e:
                break

            # Load frames as gray img
            frame_0_gray = load_gray_img(frame_0_path)
            frame_1_gray = load_gray_img(frame_1_path)

            # Extract longest contour and save it
            contour_0 = extract_longest_contour(frame_0_gray, closing_kernel_size, cv2.CHAIN_APPROX_TC89_KCOS)
            np.save(os.path.join(contours_folder_path, frame[:5]), contour_0)
            
            # Calculate optical flow to get contour_1
            contour_1, st, err = cv2.calcOpticalFlowPyrLK(frame_0_gray, frame_1_gray, 
                                                          contour_0, None, **lk_params)
            contour_1_real = extract_longest_contour(frame_0_gray, closing_kernel_size, cv2.CHAIN_APPROX_NONE)
            
            # Match contour 1 on contour 1 real
            contour_1_final = match_points(contour_1, contour_1_real)
            
            # Get translation and save it
            translation_0_1 = get_translations(contour_0, contour_1)
            translation_0_1_final = get_translations(contour_0, contour_1_final)
            
            correction = np.mean(np.linalg.norm(translation_0_1-translation_0_1_final, axis=1))
            #print('\t-> Mean correction after backpropagating on real contour:', np.mean(translation_0_1-translation_0_1_final))
            
            np.save(os.path.join(translations_folder_path, frame[:5]), translation_0_1)

            # Save annotation with contour and translation
            annotation = cv2.imread(os.path.join(annotations_folders_path, sequence, frame))
            save_annotation_with_contour_and_translation(annotation, contour_0, translation_0_1_final, 
                os.path.join(translations_folder_path, frame[:5] + '.png'))         

In [11]:
create_contours_and_translations(ANNOTATIONS_FOLDERS_PATH,
                                 CONTOURS_FOLDERS_PATH,
                                 TRANSLATIONS_FOLDERS_PATH,
                                 CLOSING_KERNEL_SIZE)

#0: bear
	#0: 00000.png
	#1: 00001.png
	#2: 00002.png
	#3: 00003.png
	#4: 00004.png
	#5: 00005.png
	#6: 00006.png
	#7: 00007.png
	#8: 00008.png
	#9: 00009.png
	#10: 00010.png
	#11: 00011.png
	#12: 00012.png
	#13: 00013.png
	#14: 00014.png
	#15: 00015.png
	#16: 00016.png
	#17: 00017.png
	#18: 00018.png
	#19: 00019.png
	#20: 00020.png
	#21: 00021.png
	#22: 00022.png
	#23: 00023.png
	#24: 00024.png
	#25: 00025.png
	#26: 00026.png
	#27: 00027.png
	#28: 00028.png
	#29: 00029.png
	#30: 00030.png
	#31: 00031.png
	#32: 00032.png
	#33: 00033.png
	#34: 00034.png
	#35: 00035.png
	#36: 00036.png
	#37: 00037.png
	#38: 00038.png
	#39: 00039.png
	#40: 00040.png
	#41: 00041.png
	#42: 00042.png
	#43: 00043.png
	#44: 00044.png
	#45: 00045.png
	#46: 00046.png
	#47: 00047.png
	#48: 00048.png
	#49: 00049.png
	#50: 00050.png
	#51: 00051.png
	#52: 00052.png
	#53: 00053.png
	#54: 00054.png
	#55: 00055.png
	#56: 00056.png
	#57: 00057.png
	#58: 00058.png
	#59: 00059.png
	#60: 00060.png
	#61: 00061.png
	#62: 000